In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from operator import add

In [2]:
class QualityState(TypedDict):
    text: str
    quality_score: float
    iteration: int
    max_iterations: int
    improvements: Annotated[list[str], add]

def generate_text(state: QualityState) -> QualityState:
    """Generate or improve text"""
    if state["iteration"] == 0:
        # First generation
        text = "This is initial text."
    else:
        # Improve existing text
        text = state["text"] + " Improved version."
    
    return {
        "text": text,
        "iteration": state["iteration"] + 1,
        "improvements": [f"Iteration {state['iteration'] + 1} completed"]
    }

def evaluate_quality(state: QualityState) -> QualityState:
    """Evaluate text quality"""
    # Simulate quality scoring (in real app, use actual metrics)
    quality = min(0.5 + (state["iteration"] * 0.15), 1.0)
    
    return {"quality_score": quality}

def check_quality(state: QualityState) -> Literal["improve", "done"]:
    """Decide if we need more iterations"""
    # Exit if max iterations reached
    if state["iteration"] >= state["max_iterations"]:
        return "done"
    
    # Exit if quality is good enough
    if state["quality_score"] >= 0.8:
        return "done"
    
    # Otherwise, improve more
    return "improve"

# Build graph
graph = StateGraph(QualityState)
graph.add_node("generate", generate_text)
graph.add_node("evaluate", evaluate_quality)

graph.add_edge(START, "generate")
graph.add_edge("generate", "evaluate")
graph.add_conditional_edges(
    "evaluate",
    check_quality,
    {
        "improve": "generate",  # Loop back!
        "done": END
    }
)

workflow = graph.compile()

# Run
result = workflow.invoke({
    "text": "",
    "quality_score": 0.0,
    "iteration": 0,
    "max_iterations": 10,
    "improvements": []
})

print(f"Final quality: {result['quality_score']}")
print(f"Total iterations: {result['iteration']}")
print(f"Final text: {result['text']}")

Final quality: 0.8
Total iterations: 2
Final text: This is initial text. Improved version.
